
<font size="5">Data Understanding</font>

Questa fase consiste nell’ <b> identificazione </b>, <b>collezione</b> e <b>analisi</b> dei dataset che possono portare al raggiungimento degli obiettivi.
Inoltre in questa fase ci occuperemo anche della <b>data visualization </b> e  <b> data quality </b>.

La scelta proggettuale è stata quella di generare personalemente il dataset a partire da risorse trovate online. Gli Urls (phishing e legitimate) sono stati reperiti da diverse fonti ad esempio: PhishTank, Openphish e Kaggle.

<br>
Importiamo le librerie utilizzate in questa fase

In [ ]:
#libreria necessaria per utilizzare funzioni definite in notebook diversi
#!pip install import-ipynb
import import_ipynb

import csv
import pandas as pd
from pathlib import Path  
import os
from os import path
import generateDataset
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import sklearn

<br>
<b><font size="4">Identificazione dei dataset</font></b>

Nello specifico sono stati identificati 5 dataset.
Due di questi vengono utilizzati per estrarre tutti gli <b>url leggittimi</b> (legitimate) e sono i seguenti:
- dataset_phishing_and_legitimate.csv
- dataset_phishing_and_legitimate2.csv

mentre gli altri 3 dataset vengono utilizzati per estrarre tutti gli <b> url phishing </b> e sono i seguenti:
- dataset_phishing.csv
- dataset_phishing2.txt
- openphish.txt

<br>
<b><font size="4">Collezione dei dataset</font></b>

<b><font size="2">Eliminazione delle colonne inutili e concatenazione dei dataset</font></b>

Come detto precedentemente la scelta proggettuale è quella di generare all'interno del progetto il dataset.
Dunque quello che ci interessa nei dataset scaricati sono solo gli Url, gli altri dati possono essere scartati.

Nel codice seguente andremo ad eliminare tutte le informazioni superflue e creeremo un unico dataset contenete i soli url e la loro classe

In [ ]:
# PRIMO DATASET LEGITIMATE
# caricamento del dataset ed estrazione di tutte le istanze legitimate da dataset_phishing_and_legitimate.csv
data = pd.read_csv("./dataset/dataset_phishing_and_legitimate.csv")
legitimate_dataset1 = data.drop(data[data.status == "phishing"].index)

#eliminazione delle feature(colonne)
legitimate_dataset1 = legitimate_dataset1.drop(legitimate_dataset1.iloc[:, 1:88],axis = 1)



# SECONDO DATASET LEGITIMATE
# caricamento del secondo dataset ed estrazione di tutte le istanze legitimate da dataset_phishing_and_legitimate.csv

data2 = pd.read_csv("./dataset/dataset_phishing_and_legitimate2.csv")
legitimate_dataset2 = data2.drop(data2[(data2['status'] == "phishing") | (data2['status'] == "malware") | (data2['status'] == "defacement") ].index)

legitimate_dataset2 = legitimate_dataset2.head(60000)

# shuffle dei dati
legitimate_dataset2 =sklearn.utils.shuffle(legitimate_dataset2)

#cambiamo benign con legitimate per avere dei valori consistenti
legitimate_dataset2['status'] = legitimate_dataset2['status'].replace('benign', 'legitimate', regex=True)


#concatenzaione dei due dataset Legitimate
frame = [legitimate_dataset1, legitimate_dataset2]
legitimate_dataset = pd.concat(frame)


# PRIMO DATASET PHISHING
# caricamento del dataset
phishing_dataset1 = pd.read_csv("./dataset/dataset_phishing.csv")

#eliminazione della colonna id
phishing_dataset1 = phishing_dataset1.drop(phishing_dataset1.columns[[0]],axis = 1)

#eliminazione delle colonne restanti
phishing_dataset1 = phishing_dataset1.drop(phishing_dataset1.iloc[:, 1:7],axis = 1)
phishing_dataset1.insert(1, "status", "phishing", True)


# SECONDO DATASET PHISHING
# caricamento del dataset
phishing_dataset2  = pd.read_csv('./dataset/openphish.txt', sep=" ",header=None )
phishing_dataset2.columns = ["url"]
phishing_dataset2.insert(1, "status", "phishing", True)


# TERZO DATASET PHISHING
phishing_dataset3  = pd.read_csv('./dataset/dataset_phishing2.txt', sep=" ",header=None )
phishing_dataset3.columns = ["url"]
phishing_dataset3.insert(1, "status", "phishing", True)

# shuffle dei dati
phishing_dataset3 =sklearn.utils.shuffle(phishing_dataset3)

#estraiamo le prime 30000 istanze da phishing_dataset3
phishing_dataset3 = phishing_dataset3.head(50163) 


#concateniamo i 3 dataset
frames = [phishing_dataset1,phishing_dataset2,phishing_dataset3]
phishing_dataset = pd.concat(frames)


#concateniamo i 2 dataset, avendo cosi un unico dataset
frames = [legitimate_dataset, phishing_dataset]
dataset = pd.concat(frames)

#eliminiamo "www." da tutte le istanze del dataset
dataset['url'] = dataset['url'].replace('www.', '', regex=True)

# shuffle dei dati
dataset=sklearn.utils.shuffle(dataset)

<br>
Il risultato di tali operazioni ci porterà alla creazione di un unico dataset con più di 128000 entry, un esempio:

In [ ]:
dataset.head()

<br>
<b><font size="4">Generazione del dataset</font></b>


<br>
Il dataset che adesso abbiamo a disposizione è formato unicamente da 2 colonne, ovvero l'url e lo status. 
Per creare un modello di Machine Learning abbiamo bisogno di estrarre delle caratteristiche (feature) dagli Url a disposizione.
In questa fase ci occuperemo di estrarre le feature da ogni url.

Il modulo che si occupa della creazione del dataset con le feature è il modulo <i>"generateDataset"</i>, mentre le funzioni che si occupano dell'estrazione delle feature vera e propria si trovano in <i>"featureExtraction"</i>

Il progetto è consegnato con il dataset finale (ovvero già provvisto di feature) già generato, il quale è salvato nel file "finalDataset.csv". Tuttavia nel caso si volesse ri-generare il dataset finale, basta eliminare il file "finalDataset.csv", l'operazione potrebbe richiedere diversi minuti.

In [ ]:
file = "finalDataset.csv"
if path.exists(file) and os.path.getsize(file) != 0:
    #legge da finalDataset.csv
    print("Leggo da finalDataset.csv")
    finalDataset = pd.read_csv(file)
else:
    #crea il dataset
    print("creo il dataset")
    finalDataset = generateDataset.generate(dataset,file)  


<br>
Il risultato dell'estrazione delle feature, porta alla creazione del dataset su cui andremo a lavorare ovvero:

In [ ]:
finalDataset.head()

<br><b><font size="4">Analisi del dataset</font></b>

Dopo aver estratto gli Url dai vari dataset scaricati, e dopo aver estratto le feature da tutti gli Url, adesso andremo ad analizzare il dataset 

In [ ]:
#esaminiamo i dati 
finalDataset.info()

Avremo dunque 128308 entry, e 34 colonne di cui 32 sono le feature estratte.
<br>

<br><br>
<font size="4">Presenza duplicati</font>
<br>

Controlliamo la presenza di duplicati nel dataset

In [ ]:
sum = finalDataset.duplicated().sum()
print(sum)

Avremo dunque 197 duplicati nel dataset e nello specifico sono gli url:

In [ ]:
duplicateRows = finalDataset[finalDataset.duplicated()]
print (duplicateRows["url"])

<br><br>
<font size="4">Bilanciamento dei dati</font></b>

Andiamo a visualizzare il bilanciamento dei dati

In [ ]:
count = finalDataset.label.value_counts()

data = [count["legitimate"], count["phishing"]]
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

plt.pie(data, labels = labels,colors = colors,autopct='%.0f%%')

print("Numero url legitimate: ",count["legitimate"])
print("Numero url phishing: " ,count["phishing"])
plt.title("Bilanciamento dati")
plt.show()

Avendo creato il dataset personalmente, lo abbiamo creato appositamente bilanciato<br><br>



<font size="4">Dati mancanti</font></b>

Controlliamo se abbiamo valori null all'interno del dataset

In [ ]:
finalDataset.isnull().sum()

Abbiamo 54422 entry con valore null per la feature "https" <br><br>

<font size="4">Ricerca outlier</font></b>

In questa fase andremo a ricercare possibili outlier all'interno del dataset, utilizzeremo un box plot per osservare possibili outlier. <br>

Utilizzo la feature "urlLenght" in quanto da questa dipendono la maggior parte delle feature, come ad esempio lenghtSub, lenghtPath, numLetters, ecc... Mentre non considero le altre feature in quanto sono valori booleani.

In [ ]:
finalDataset.urlLenght.describe()

Già in questa fase possiamo notare che c'è almeno un elemento nel dataset che ha la feature "urlLenght" pari a 5787, mentre la media di tutti gli elementi è 56. Questo chiaramente indica la presenza di outlier

In [ ]:
fig = px.box(finalDataset, x = "label", y="urlLenght")
fig.show()

Abbiamo conferma della presenza di outlier anche grazie a questo grafico.

Guardando il grafico non riusciamo ad ottenre molte altre informazioni, tuttavia possiamo notare che ci sono altri outlier.

Utilizzeremo l'IQR method per identificare gli altri outlier.

In [ ]:
def detectOutliers(data):
    quartile_1, quartile_3 = np.percentile(data["urlLenght"], [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    print("Lower bound:", lower_bound)
    print("Upper bound:", upper_bound)
    outliers = data[(data["urlLenght"] <= lower_bound) | (data["urlLenght"] >= upper_bound)] 
    print("Numero di outlier: ", outliers.shape[0])
    
    count = outliers.label.value_counts()
    print("numero outlier legitimate:",count["legitimate"])
    print("numero outlier phishing:",count["phishing"])
    
    return lower_bound,upper_bound

detectOutliers(finalDataset)

Possiamo concludere che tutti gli Url che hanno la feature "urlLenght" che fuoriesce dai bound (Upper bound o Lower bound) sono da considerare outlier. Utilizzando il IQR method possiamo identificare ben 7123 outlier. Ovvero il 5% dell'intero dataset.
 

Eliminazione degli outlier:

In [ ]:
lower_bound,upper_bound = detectOutliers(finalDataset)

#aggiorno il dataset scartando gli outliers
finalDataset = finalDataset[(finalDataset["urlLenght"] >= lower_bound) & (finalDataset["urlLenght"] <= upper_bound)]

#plotto il dataset aggiornato
fig = px.box(finalDataset, x = "label", y="urlLenght")
fig.show()

<br>
<b><font size="4">Data visualization</font></b>

in questa fase andremo ad <b>esplorare i dati</b> ed a <b>visualizzarli</b>. Inoltre andremo ad identificare le relazioni che ci sono fra i dati.

Quello che andremo a fare in questo caso è andare a raffigurare i grafici in cui rappresentiamo sull'asse delle x la feature, mentre sull'asse delle y il numero di istanze. 
Queste informazioni potranno tornarci utili nella fase di "Data preparation".

In questa prima parte di grafici, andremo a valutare tutte le feature che hanno un valore numerico

<br><br>
<b><font size="3">Lunghezza del sottodominio (lenghtSub)</font></b> 

In [ ]:

legitimate = finalDataset[finalDataset['label'] == "legitimate"]
phishing = finalDataset[finalDataset['label'] == "phishing"]

plt.hist([legitimate['lenghtSub'], phishing['lenghtSub']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')

plt.ylabel('numero istanze')
plt.xlabel('lunghezza sottodominio')
plt.title('distribuzione')
plt.show()

<br><br><b><font size="3">Lunghezza dell' Url (urlLenght)</font></b> 

In [ ]:
plt.hist([legitimate['urlLenght'], phishing['urlLenght']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('lunghezza sottodominio')
plt.title('lunghezza url')
plt.show()

<br><br><b><font size="3">Numero di cifre all'interno dell'Url (numDigits)</font></b> 

In [ ]:
plt.hist([legitimate['numDigits'], phishing['numDigits']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('umero cifre nel url')
plt.title('distribuzione')
plt.show()

<br><br><b><font size="3">Numero di lettere all'interno dell'Url (numLetters)</font></b> 

In [ ]:
plt.hist([legitimate['numLetters'], phishing['numLetters']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('numero lettere nell url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di '&' nell'Url (numAmpersand)</font></b> 

In [ ]:
plt.hist([legitimate['numAmpersand'], phishing['numAmpersand']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di & nell Url')
plt.title('distribuzione')
plt.show()

 
<br><br> <b><font size="3">Numero di '-' nell'Url (numDash)</font></b> 

In [ ]:
plt.hist([legitimate['numDash'], phishing['numDash']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di - nell Url')
plt.title('distribuzione')
plt.show()


<br><br> <b><font size="3">Numero di '@' nell'Url (numAt)</font></b>


In [ ]:
plt.hist([legitimate['numAt'], phishing['numAt']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di @ nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di '?' nell'Url (numQM)</font></b>


In [ ]:
plt.hist([legitimate['numQM'], phishing['numQM']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di ? nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di '|' nell'Url (numVS)</font></b>

In [ ]:
plt.hist([legitimate['numVS'], phishing['numVS']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di | nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di '=' nell'Url (numEqual)</font></b>

In [ ]:
plt.hist([legitimate['numEqual'], phishing['numEqual']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di = nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di '_' nell'Url (numUnderscore)</font></b>

In [ ]:
plt.hist([legitimate['numUnderscore'], phishing['numUnderscore']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di _ nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di '˜' nell'Url (numTilde)</font></b>

In [ ]:
plt.hist([legitimate['numTilde'], phishing['numTilde']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di ˜ nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di '%' nell'Url (numPercente)</font></b>

In [ ]:
plt.hist([legitimate['numPercente'], phishing['numPercente']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di % nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di '*' nell'Url (numAsterisc)</font></b>

In [ ]:
plt.hist([legitimate['numAsterisc'], phishing['numAsterisc']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di * nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di '$' nell'Url (numDollar)</font></b> 

In [ ]:
plt.hist([legitimate['NumDollar'], phishing['NumDollar']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di $ nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di ';' nell'Url (numSC)</font></b>

In [ ]:
plt.hist([legitimate['numSC'], phishing['numSC']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di ; nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di ':' nell'Url (numColons)</font></b>

In [ ]:
plt.hist([legitimate['numColons'], phishing['numColons']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di : nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di ''' nell'Url (numSQ)</font></b>

In [ ]:
plt.hist([legitimate['numSQ'], phishing['numSQ']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di \' nell Url')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Numero di sottodomini nell'Url (numberSub)</font></b>

In [ ]:
plt.hist([legitimate['numberSub'], phishing['numberSub']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Numero di sottodomini')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Lunghezza del dominio nell'Url (lenghtDom)</font></b>

In [ ]:
plt.hist([legitimate['lenghtDom'], phishing['lenghtDom']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Lunghezza del dominio')
plt.title('distribuzione')
plt.show()

<br><br> <b><font size="3">Lunghezza del path nell'Url (lenghtPath)</font></b>

In [ ]:
plt.hist([legitimate['lenghtPath'], phishing['lenghtPath']], label=['legitimate', 'phishing'],color=['royalblue','red'])
plt.legend(loc='upper right')
plt.ylabel('numero istanze')
plt.xlabel('Lungezza path')
plt.title('distribuzione')
plt.show()

<br> 
I prossimi grafici rappresentano tutte le feature che hanno valore booleano

<b><font size="3"> Dettagli riguardo la feature: dash</font></b> 

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(12, 10))

#estrai il numero di url che hanno il dash (per tutti gli url phishing)
phishingDash = phishing[phishing["dash"] == 1]
phishingNoDash = phishing[phishing["dash"] == 0]

#imposta i dati da plottare 
data = [phishingDash.shape[0], phishingNoDash.shape[0]]
labels = ['Dash', 'No Dash']
colors = ["salmon","red"]

#plotta i dati
ax1.set_title('Dash in phishing Url')
ax1.pie(data, labels = labels,colors = colors,autopct='%.0f%%')


#estrai il numero di url che hanno il dash (per tutti gli url legitimate)
legitimateDash = legitimate[legitimate['dash'] == 1]
legitimateNoDash = legitimate[legitimate['dash'] == 0]

#imposta i dati da plottare 
data = [legitimateDash.shape[0], legitimateNoDash.shape[0]]
labels = ['Dash', 'No Dash']
colors = ["lightskyblue","royalblue"]

#plotta i dati
ax2.set_title('Dash in legitimate Url')
ax2.pie(data, labels = labels,colors = colors,autopct='%.0f%%')


urlDash = finalDataset[finalDataset["dash"] == 1]

count = urlDash.label.value_counts()

data = [count["legitimate"], count["phishing"]]
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

#Nel grafico ax3, andiamo a graficare TUTTI gli Url che hanno usano il dash, nello specifico andremo a suddividere il grafico in phishing e legitimate 
ax3.set_title('Url using dash')
ax3.pie(data, labels = labels,colors = colors, autopct='%.0f%%')

plt.show()

print("Url legitimate using dash",count["legitimate"])
print("Url phishing using dash",count["phishing"])



<br><br><b><font size="3"> Dettagli rigurdo la feature: ShortiningServ</font></b> 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(11, 10))

#estrai il numero di url che hanno il dash (per tutti gli url phishing)
phishingShortServ = phishing[phishing["ShortiningServ"] == 1]
phishingNoShortServ = phishing[phishing["ShortiningServ"] == 0]

#imposta i dati da plottare 
data = [phishingShortServ.shape[0], phishingNoShortServ.shape[0]]
labels = ['Using Short. Serv.', 'No Short. Serv.']
colors = ["salmon","red"]

#plotta i dati
ax1.set_title('Phshing Url')
ax1.pie(data, labels = labels,colors = colors,autopct='%.0f%%')



#estrai il numero di url che hanno il dash (per tutti gli url legitimate)
legitimateShortServ = legitimate[legitimate['ShortiningServ'] == 1]
legitimateNoShortServ = legitimate[legitimate['ShortiningServ'] == 0]

#imposta i dati da plottare 
data = [legitimateShortServ.shape[0], legitimateNoShortServ.shape[0]]
colors = ["lightskyblue","royalblue"]

#plotta i dati
ax2.set_title('Legitimate Url')
ax2.pie(data, labels = labels,colors = colors,autopct='%.0f%%')
fig.tight_layout(pad=6)
plt.show()


fig, ax3 = plt.subplots(1,1, figsize=(3, 10))
urlShortiningServ = finalDataset[finalDataset["ShortiningServ"] == 1]
count = urlShortiningServ.label.value_counts()


#controllo sul numero di istanze legitimate o phishing
if 'legitimate' not in count:
    numLegit = 0
else:
    numLegit = count["legitimate"]

if 'phishing' not in count:
    numPhish = 0
else:
    numPhish = count["phishing"]


data = [numLegit, numPhish]
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

#Nel grafico ax3, andiamo a graficare TUTTI gli Url che hanno usano il dash, nello specifico andremo a suddividere il grafico in phishing e legitimate 
ax3.set_title('Url using Shortening Services')
ax3.pie(data, labels = labels,colors = colors,autopct='%.0f%%')
plt.show()


print("Url legitimate using Shortening Services",numLegit)
print("Url phishing using Shortening Services",numPhish)


<br><br><b><font size="3"> Dettagli rigurdo la feature: httpsDomSub</font></b>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(15, 10))

phishing1 = phishing[phishing["httpsDomSub"] == 1]
phishing0 = phishing[phishing["httpsDomSub"] == 0]

#imposta i dati da plottare 
data = [phishing1.shape[0], phishing0.shape[0]]
labels = ['Yes Https', 'No Https']
colors = ["salmon","red"]

#plotta i dati
ax1.set_title('Phshing Url')
ax1.pie(data, labels = labels,colors = colors,autopct='%.0f%%')


legitimate1 = legitimate[legitimate['httpsDomSub'] == 1]
legitimate0 = legitimate[legitimate['httpsDomSub'] == 0]

#imposta i dati da plottare 
data = [legitimate1.shape[0], legitimate0.shape[0]]
colors = ["lightskyblue","royalblue"]

#plotta i dati
ax2.set_title('Legitimate Url')
ax2.pie(data, labels = labels,colors = colors,autopct='%.0f%%')


final = finalDataset[finalDataset["httpsDomSub"] == 1]
count = final.label.value_counts()

#controllo sul numero di istanze legitimate o phishing
if 'legitimate' not in count:
    numLegit = 0
else:
    numLegit = count["legitimate"]

if 'phishing' not in count:
    numPhish = 0
else:
    numPhish = count["phishing"]

data = [numLegit,numPhish] 
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

ax3.set_title('Url with https in domain')
ax3.pie(data, labels = labels,colors = colors,autopct='%.0f%%')
fig.tight_layout(pad=6)
plt.show()


print("Numero di url legitimate con https nel dominio", numLegit)
print("Numero di url phishing con https nel dominio", numPhish)



<br><br><b><font size="3"> Dettagli rigurdo la feature: port</font></b>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(15, 10))

phishing1 = phishing[phishing["port"] == 1]
phishing0 = phishing[phishing["port"] == 0]

#imposta i dati da plottare 
data = [phishing1.shape[0], phishing0.shape[0]]
labels = ['port in Url', 'No port in Url']
colors = ["salmon","red"]

#plotta i dati
ax1.set_title('Phshing Url')
ax1.pie(data, labels = labels,colors = colors,autopct='%.0f%%')


legitimate1 = legitimate[legitimate['port'] == 1]
legitimate0 = legitimate[legitimate['port'] == 0]

#imposta i dati da plottare 
data = [legitimate1.shape[0], legitimate0.shape[0]]
colors = ["lightskyblue","royalblue"]

#plotta i dati
ax2.set_title('Legitimate Url')
ax2.pie(data, labels = labels,colors = colors,autopct='%.0f%%')



final = finalDataset[finalDataset["port"] == 1]
count = final.label.value_counts()

#controllo sul numero di istanze legitimate o phishing
if 'legitimate' not in count:
    numLegit = 0
else:
    numLegit = count["legitimate"]

if 'phishing' not in count:
    numPhish = 0
else:
    numPhish = count["phishing"]

data = [numLegit,numPhish]    
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

ax3.set_title('Url with port in url')
ax3.pie(data, labels = labels,colors = colors,autopct='%.0f%%')
fig.tight_layout(pad=4)

plt.show()

print("Numero di url legitimate con port nell url", numLegit)
print("Numero di url phishing con port nell url", numPhish)


<br><br><b><font size="3"> Dettagli rigurdo la feature: checkPath</font></b>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(15, 10))

phishing1 = phishing[phishing["checkPath"] == 1]
phishing0 = phishing[phishing["checkPath"] == 0]

#imposta i dati da plottare 
data = [phishing1.shape[0], phishing0.shape[0]]
labels = ['Contain txt,exe,js', 'No txt,exe,js']
colors = ["salmon","red"]

#plotta i dati
ax1.set_title('Phshing Url')
ax1.pie(data, labels = labels,colors = colors,autopct='%.0f%%')


legitimate1 = legitimate[legitimate['checkPath'] == 1]
legitimate0 = legitimate[legitimate['checkPath'] == 0]

#imposta i dati da plottare 
data = [legitimate1.shape[0], legitimate0.shape[0]]
colors = ["lightskyblue","royalblue"]

#plotta i dati
ax2.set_title('Legitimate Url')
ax2.pie(data, labels = labels,colors = colors,autopct='%.0f%%')



final = finalDataset[finalDataset["checkPath"] == 1]
count = final.label.value_counts()

if 'legitimate' not in count:
    numLegit = 0
else:
    numLegit = count["legitimate"]

if 'phishing' not in count:
    numPhish = 0
else:
    numPhish = count["phishing"]
    
data = [numLegit,numPhish]  
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

ax3.set_title('Url with .txt, .exe, .js')
ax3.pie(data, labels = labels,colors = colors,autopct='%.0f%%')
fig.tight_layout(pad=4)
plt.show()

print("Numero di url legitimate con parole come txt,exe,js ", numLegit)
print("Numero di url phishing con come txt,exe,js", numPhish)


<br><br><b><font size="3"> Dettagli rigurdo la feature: PunyCode</font></b>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(15, 10))

phishing1 = phishing[phishing["PunyCode"] == 1]
phishing0 = phishing[phishing["PunyCode"] == 0]

#imposta i dati da plottare 
data = [phishing1.shape[0], phishing0.shape[0]]
labels = ['Use PunyCode', 'Not using PunyCode']
colors = ["salmon","red"]

#plotta i dati
ax1.set_title('Phshing Url')
ax1.pie(data, labels = labels,colors = colors,autopct='%.0f%%')


legitimate1 = legitimate[legitimate['PunyCode'] == 1]
legitimate0 = legitimate[legitimate['PunyCode'] == 0]

#imposta i dati da plottare 
data = [legitimate1.shape[0], legitimate0.shape[0]]
colors = ["lightskyblue","royalblue"]

#plotta i dati
ax2.set_title('Legitimate Url')
ax2.pie(data, labels = labels,colors = colors,autopct='%.0f%%')



final = finalDataset[finalDataset["PunyCode"] == 1]
count = final.label.value_counts()

if 'legitimate' not in count:
    numLegit = 0
else:
    numLegit = count["legitimate"]

if 'phishing' not in count:
    numPhish = 0
else:
    numPhish = count["phishing"]
    
data = [numLegit,numPhish]  
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

ax3.set_title('Url using PunyCode')
ax3.pie(data, labels = labels,colors = colors,autopct='%.0f%%')
fig.tight_layout(pad=4)
plt.show()


print("Numero di url legitimate con PunyCode ", numLegit)
print("Numero di url phishing con PunyCode", numPhish)
    

<br><br><b><font size="3"> Dettagli rigurdo la feature: suspWords</font></b>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(16, 20))

phishing1 = phishing[phishing["suspWords"] == 1]
phishing0 = phishing[phishing["suspWords"] == 0]

#imposta i dati da plottare 
data = [phishing1.shape[0], phishing0.shape[0]]
labels = ['Using suspWords', 'Not using suspWords']
colors = ["salmon","red"]

#plotta i dati
ax1.set_title('Phshing Url')
ax1.pie(data, labels = labels,colors = colors,autopct='%.0f%%')


legitimate1 = legitimate[legitimate['suspWords'] == 1]
legitimate0 = legitimate[legitimate['suspWords'] == 0]

#imposta i dati da plottare 
data = [legitimate1.shape[0], legitimate0.shape[0]]
colors = ["lightskyblue","royalblue"]

#plotta i dati
ax2.set_title('Legitimate Url')
ax2.pie(data, labels = labels,colors = colors,autopct='%.0f%%')



final = finalDataset[finalDataset["suspWords"] == 1]
count = final.label.value_counts()

if 'legitimate' not in count:
    numLegit = 0
else:
    numLegit = count["legitimate"]

if 'phishing' not in count:
    numPhish = 0
else:
    numPhish = count["phishing"]
    
data = [numLegit,numPhish]  
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

ax3.set_title('Url using suspWords')
ax3.pie(data, labels = labels,colors = colors,autopct='%.0f%%')
fig.tight_layout(pad=3)
plt.show()

print("Numero di url legitimate con suspWords ", numLegit)
print("Numero di url phishing con suspWords", numPhish)


<br><br><b><font size="3"> Dettagli rigurdo la feature: https</font></b>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(15, 10))

phishing1 = phishing[phishing["https"] == 1]
phishing0 = phishing[phishing["https"] == 0]

#imposta i dati da plottare 
data = [phishing1.shape[0], phishing0.shape[0]]
labels = ['Using https', 'Not using https']
colors = ["salmon","red"]

#plotta i dati
ax1.set_title('Phshing Url')
ax1.pie(data, labels = labels,colors = colors,autopct='%.0f%%')


legitimate1 = legitimate[legitimate['https'] == 1]
legitimate0 = legitimate[legitimate['https'] == 0]

#imposta i dati da plottare 
data = [legitimate1.shape[0], legitimate0.shape[0]]
colors = ["lightskyblue","royalblue"]

#plotta i dati
ax2.set_title('Legitimate Url')
ax2.pie(data, labels = labels,colors = colors,autopct='%.0f%%')


final = finalDataset[finalDataset["https"] == 1]
count = final.label.value_counts()

if 'legitimate' not in count:
    numLegit = 0
else:
    numLegit = count["legitimate"]

if 'phishing' not in count:
    numPhish = 0
else:
    numPhish = count["phishing"]

data = [numLegit,numPhish]  
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

ax3.set_title('Url using https')
ax3.pie(data, labels = labels,colors = colors,autopct='%.0f%%')
fig.tight_layout(pad=6)
plt.show()

print("Numero di url legitimate che usano https", numLegit)
print("Numero di url phishing che usano https", numPhish)

Questo grafico fa notare un dettaglio interessante, nello specifico nel grafico "Legitimate Url" possiamo notare che solo il 27% degli Url legittimi utilizzano il protocollo https, mentre il restante degli url legittimi (ovvero il 73%) utilizza il protocollo http.Chiaramente ricordiamo che <b>non stiamo considerando tutti gli Url</b> in quanto molti di questi non hanno nessuna informazione riguardo il protocollo. 

Anche questo è un dettaglio che utilizzaremo in fase di "Data preparation"

<br><br><b><font size="3"> Dettagli rigurdo la feature: email</font></b>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(18, 10))

phishing1 = phishing[phishing["email"] == 1]
phishing0 = phishing[phishing["email"] == 0]

#imposta i dati da plottare 
data = [phishing1.shape[0], phishing0.shape[0]]
labels = ['Using email', 'Not using email']
colors = ["salmon","red"]

#plotta i dati
ax1.set_title('Phshing Url')
ax1.pie(data, labels = labels,colors = colors,autopct='%.0f%%')

legitimate1 = legitimate[legitimate['email'] == 1]
legitimate0 = legitimate[legitimate['email'] == 0]

#imposta i dati da plottare 
data = [legitimate1.shape[0], legitimate0.shape[0]]
colors = ["lightskyblue","royalblue"]

#plotta i dati
ax2.set_title('Legitimate Url')
ax2.pie(data, labels = labels,colors = colors,autopct='%.0f%%')



final = finalDataset[finalDataset["email"] == 1]
count = final.label.value_counts()

if 'legitimate' not in count:
    numLegit = 0
else:
    numLegit = count["legitimate"]

if 'phishing' not in count:
    numPhish = 0
else:
    numPhish = count["phishing"]

data = [numLegit,numPhish]  
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

ax3.set_title('Url using email')
ax3.pie(data, labels = labels,colors = colors,autopct='%.0f%%')
fig.tight_layout(pad=6)
plt.show()


print("Numero di url legitimate che usano email", numLegit)
print("Numero di url phishing che usano email", numPhish)

<br><br><b><font size="3"> Dettagli rigurdo la feature: IP</font></b>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(15, 10))

phishing1 = phishing[phishing["IP"] == 1]
phishing0 = phishing[phishing["IP"] == 0]

#imposta i dati da plottare 
data = [phishing1.shape[0], phishing0.shape[0]]
labels = ['Using IP', 'Not using IP']
colors = ["salmon","red"]

#plotta i dati
ax1.set_title('Phshing Url')
ax1.pie(data, labels = labels,colors = colors,autopct='%.0f%%')

legitimate1 = legitimate[legitimate['IP'] == 1]
legitimate0 = legitimate[legitimate['IP'] == 0]

#imposta i dati da plottare 
data = [legitimate1.shape[0], legitimate0.shape[0]]
colors = ["lightskyblue","royalblue"]

#plotta i dati
ax2.set_title('Legitimate Url')
ax2.pie(data, labels = labels,colors = colors,autopct='%.0f%%')



final = finalDataset[finalDataset["IP"] == 1]
count = final.label.value_counts()

#controllo sul numero di istanze legitimate o phishing 
if 'legitimate' not in count:
    numLegit = 0
else:
    numLegit = count["legitimate"]

if 'phishing' not in count:
    numPhish = 0
else:
    numPhish = count["phishing"]

data = [numLegit,numPhish]  
labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

ax3.set_title('Url using IP')
ax3.pie(data, labels = labels,colors = colors,autopct='%.0f%%')
fig.tight_layout(pad=6)
plt.show()

print("Numero di url legitimate che usano IP", numLegit)
print("Numero di url phishing che usano IP", numPhish)
    


<br><br>
<font size="5">Data preparation</font>

Questa fase ha l’obbiettivo di passare dal dataset iniziale ad un dataset utilizzabile da un algoritmo di ML.

Nello specifico in questa fase ci soffermeremo alla fase di:
 - Data cleaning
 - Feature scaling
 - Feature selection
 - Data balancing
 


<br>
<font size="4"> <b>Data cleaning </b> </font>


<br>
Abbiamo visto che avremo 197 duplicati, andremo ad eliminare i duplicati.

Eliminazione dei duplicati:

In [ ]:
finalDataset = finalDataset.drop_duplicates()

<br>
<font size="4"> Data imputation </font>
<br>

Per via dell'alto numero di valori null, e dato che tutti valori null sono dovuti alla feature "https" (maggiori dettagli nella documentazione), andremo ad eliminare la feature "https":

In [ ]:
finalDataset.drop("https", inplace=True, axis=1)

<br>
<font size="4"> <b>Feature scaling</b> </font>

In questa fase non effettuo feature scaling, ma la effettuo dopo la fase di "Modeling". 

<br>
<font size="4"> <b>Feature selection</b> </font>

Le feature con bassa varianza sono:
- numAt
- numQM
- numAmpersand
- numVS
- numUnderscore
- numTilde
- numAsterisc
- NumDollar
- numSC
- numColons
- numSQ
- numPercente

Dunque andremo ad eliminare le colonne dal dataset

In [ ]:
to_drop = ["numAt","numQM","numAmpersand", "numVS","numUnderscore","numTilde","numAsterisc","NumDollar", "numSC","numColons","numSQ","numPercente"]

#copio il dataset in modo da avere disponibile il dataset originale
fullFinalDataset = finalDataset.copy()

#eliminiamo le colonne con le feature con bassa varianza
finalDataset.drop(to_drop, inplace=True, axis=1)

<br><br>
<font size="4">Eliminazione delle feature "checkPath"</font>

La feature checkPath è stata eliminata in quanto non sembra essere una feature caratterizzante.

In [ ]:
#N.B. runnare questa cella una sola volta, se runni più volte non funziona in quanto non trova più la colonna "checkPath" 
#eliminiamo la colonna "checkPath"
finalDataset.drop("checkPath", inplace=True, axis=1)

<br>
Dopo l'eliminazione delle colonne facciamo un recap della situazione attuale: 

In [ ]:
finalDataset.info()

In questa fase abbiamo eliminato un totale di 14 colonne.

<br>
<font size="3"> <b>Data balancing</b> </font>

Come abbiamo visto nella fase precedente, il dataset è composto da un numero di istanze di Url leggittimi (51%) e Url phishing (49%), molto simili.
Di conseguenza non abbiamo necessità di bilanciare i dati.

<br><br>
<font size="5">Modeling</font>


Algoritmi presi in esame (Maggiori dettagli nella documentazione): 
- Naive Bayes
- Decision trees
- Random forests
- K-nearest neighbors

<br> importo le librerie necessarie:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from scipy.stats import zscore


<br>
<font size="3"> <b>Suddivisione dei dati (80:10:10)</b> </font>
<br>
Il trainig set sarà composto dal 80% dell'intero dataset, il validation set dal 10%, ed il test set sarà composto dal restante 10%.  

In [ ]:
#dataframe contenente tutte le feature
X = finalDataset[['IP',"urlLenght","ShortiningServ","numDash","numEqual","dash","httpsDomSub","port","numberSub","lenghtDom","lenghtSub","lenghtPath","numLetters","numDigits","email","PunyCode","suspWords","TLD"]]
#dataframe contenente unicamente la label
y = finalDataset['label']

# definiamo i training set, composto dall'80% dell'intero dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

#utilizziamo il dataset rimanente per definire il validation set ed il test set 
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

<br>Alcuni dettagli riguardo lo split dei dati

In [ ]:
countTrain = y_train.value_counts()
countValid = y_valid.value_counts()
countTest = y_test.value_counts()

dataTrain = [countTrain["legitimate"], countTrain["phishing"]]
dataValid = [countValid["legitimate"], countValid["phishing"]]
dataTest = [countTest["legitimate"], countTest["phishing"]]

print("Train set:")
print("Numero url legittimi:",countTrain["legitimate"])
print("Numero url phishing: " ,countTrain["phishing"])
print("\n")
print("Validation set:")
print("Numero url legittimi:",countValid["legitimate"])
print("Numero url phishing: " ,countValid["phishing"])
print("\n")
print("Test set:")
print("Numero url legittimi:",countTest["legitimate"])
print("Numero url phishing: " ,countTest["phishing"])

<br>Plottiamo la distribuzione delle classi

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(10, 10))

labels = ['legitimate', 'phishing']
colors = ["royalblue","red"]

ax1.set_title('Training set')
ax1.pie(dataTrain, labels = labels,colors = colors,autopct='%.0f%%')

ax2.set_title('Validation set')
ax2.pie(dataValid, labels = labels,colors = colors,autopct='%.0f%%')

ax3.set_title('Test set')
ax3.pie(dataTest, labels = labels,colors = colors,autopct='%.0f%%')

plt.show()


La distribuzione rimane uguale a quella del dataset iniziale

<br><font size="3"> <b>Test su diversi classificatori</b> </font>

Come detto precedentemente, tutti i modelli verranno allenati sul training set e valuteremo le prestazioni del modello utilizzando il validation set 

Per valutare il modello utilizzeremo il metodo <i>classification_report</i> il quale fornisce un riepilogo sulle principali metriche di valutazione (precision, recall, f1-score) e plotteremo la confusion matrix.

In questo progetto, per configurare i parametri dei modelli, utilizzeremo il Grid Search (maggiori dettagli nella documentazione del progetto).
Prima di allenare un modello andremo sempre a trovare la migliore combinazione di parametri utilizzando il Grid Search.

<br><font size="3"> <b>Naive Bayes</b> </font>

Troviamo la miglior combinazione di parametri per il modello <i>GaussianNB</i>

In [ ]:
# Definiamo la griglia dei parametri che dovremo testare sul modello
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6],
    'priors': [None, [0.1, 0.9], [0.2, 0.8], [0.3, 0.7]]
}

# definiamo il classificatore
modelGaussian = GaussianNB()

#grid search
grid_search = GridSearchCV(estimator=modelGaussian, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

#stampiamo i parametri migliori ed il miglior punteggio (in base all'accuracy)
print("Best parameters:", grid_search.best_params_)

otteremo che i parametri migliori (tra quelli definiti nella variabile) sono i seguenti:
    <i>'priors': [0.1, 0.9], 'var_smoothing': 1e-06  </i>
<br> Useremo tali parametri per allenare il modello

In [ ]:
#impostiamo il modello
modelGaussian = GaussianNB(priors = [0.1, 0.9], var_smoothing = 1e-06)

#alleniamo il modello
modelGaussian.fit(X_train, y_train)
#il modello predice i validation data
y_pred = modelGaussian.predict(X_valid)

print('Classification report')
print(classification_report(y_valid, y_pred))

print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid, y_pred)
print(cf_matrix)

<br>
<font size="3"> <b>Decision tree</b> </font>

Troviamo la miglior combinazione di parametri per il modello <i>DecisionTreeClassifier</i>

In [ ]:
param_grid = {
    'criterion': ["gini", "entropy", "log_loss"],
    'max_depth': [None, 5, 10,7,15],
    'min_samples_split': [2, 5,7,10],
    'min_samples_leaf': [1, 2, 4,6]
}

modelDT = DecisionTreeClassifier()

grid_search = GridSearchCV(estimator=modelDT, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

Otteremo questa lista di parametri:
<i>criterion = 'log_loss', max_depth = 15, min_samples_leaf = 1, min_samples_split = 2 </i>

Qundi useremo questi parametri nel nostro modello.

In [ ]:
modelDT = DecisionTreeClassifier(criterion = 'log_loss', max_depth = 15, min_samples_leaf = 1, min_samples_split = 2)
modelDT.fit(X_train, y_train)

y_pred = modelDT.predict(X_valid)

print('Classification report')
print(classification_report(y_valid, y_pred))
print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid, y_pred)
print(cf_matrix)

<br>
<font size="3"> <b>Random forests</b> </font>

Troviamo la miglior combinazione di parametri per il modello <i>RandomForestClassifier</i>

In [ ]:
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

modelRF = RandomForestClassifier()

grid_search = GridSearchCV(estimator=modelRF, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

Otteremo questa lista di parametri: <i>max_depth = None, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 200 </i>

Qundi useremo questi parametri nel nostro modello.


In [ ]:
modelRF = RandomForestClassifier(max_depth = None, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 200)

modelRF.fit(X_train, y_train)

y_pred = modelRF.predict(X_valid)

print('Classification report')
print(classification_report(y_valid, y_pred))
print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid, y_pred)
print(cf_matrix)

<br>
<font size="3"> <b>KNeighborsClassifier</b> </font>

Troviamo la miglior combinazione di parametri per il modello <i>KNeighborsClassifier</i>

In [ ]:
param_grid = {'n_neighbors': np.arange(1, 31), 'weights': ['uniform', 'distance']}

modelKnn = KNeighborsClassifier()

grid_search = GridSearchCV(estimator=modelKnn, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

Otteremo questa lista di parametri: <i> n_neighbors = 7, weight ='distance' </i>

Qundi useremo questi parametri nel nostro modello.


In [ ]:
modelKnn = KNeighborsClassifier(n_neighbors = 7, weights ='distance')

modelKnn.fit(X_train, y_train)
y_pred = modelKnn.predict(X_valid)

print('Classification report')
print(classification_report(y_valid, y_pred))

print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid, y_pred)
print(cf_matrix)

Alla fine di questa fase possiamo osservare che il modello più prestante è RandomForestClassifier con i parametri <i> max_depth = None, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 200 </i>
<br>
Dunque sarà proprio questo il modello che utilizzeremo in fase di "Evaluation".

<br>
<font size="3"> <b>Sperimentazione con suddivisione dei dati (60:20:20)</b> </font>
<br>
In questa fase andremo a sperimentare i modelli connuna diversa suddivisione del dataset.
Il trainig set sarà composto dal 60% dell'intero dataset, il validation set dal 20%, ed il test set sarà composto dal restante 20%.  

In [ ]:
#dataframe contenente tutte le feature
X60 = finalDataset[['IP',"urlLenght","ShortiningServ","numDash","numEqual","dash","httpsDomSub","port","numberSub","lenghtDom","lenghtSub","lenghtPath","numLetters","numDigits","email","PunyCode","suspWords","TLD"]]

#dataframe contenente unicamente la label
y60 = finalDataset['label']

# definiamo i training set, composto dall'60% dell'intero dataset
X_train60, X_rem60, y_train60, y_rem60 = train_test_split(X60,y60, train_size=0.6)

#utilizziamo il dataset rimanente per definire il validation set ed il test set 
X_valid60, X_test60, y_valid60, y_test60= train_test_split(X_rem60,y_rem60, test_size=0.5)

<br>
<font size="3"> Dettagli sullo plitting: </font>


In [ ]:
countTrain = y_train60.value_counts()
countValid = y_valid60.value_counts()
countTest = y_test.value_counts()

dataTrain = [countTrain["legitimate"], countTrain["phishing"]]
dataValid = [countValid["legitimate"], countValid["phishing"]]
dataTest = [countTest["legitimate"], countTest["phishing"]]

print("Train set:")
print("Numero url legittimi:",countTrain["legitimate"])
print("Numero url phishing: " ,countTrain["phishing"])
print("\n")
print("Validation set:")
print("Numero url legittimi:",countValid["legitimate"])
print("Numero url phishing: " ,countValid["phishing"])
print("\n")
print("Test set:")
print("Numero url legittimi:",countTest["legitimate"])
print("Numero url phishing: " ,countTest["phishing"])

<br><font size="3"> <b>Naive Bayes</b> </font>

In [ ]:
#impostiamo il modello
modelGaussian = GaussianNB(priors = [0.1, 0.9], var_smoothing = 1e-06)

#alleniamo il modello
modelGaussian.fit(X_train60, y_train60)
#il modello predice i validation data
y_pred = modelGaussian.predict(X_valid60)

print('Classification report')
print(classification_report(y_valid60, y_pred))

print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid60, y_pred)

<br>
<font size="3"> <b>Decision tree</b> </font>

In [ ]:
modelDT = DecisionTreeClassifier(criterion = 'entropy', max_depth = None, min_samples_leaf = 6, min_samples_split = 2 )
modelDT.fit(X_train60, y_train60)

y_pred = modelDT.predict(X_valid60)

print('Classification report')
print(classification_report(y_valid60, y_pred))

print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid60, y_pred)

<br>
<font size="3"> <b>Random forests</b> </font>

In [ ]:
modelRF = RandomForestClassifier(max_depth = None, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 200)

modelRF.fit(X_train60, y_train60)

y_pred = modelRF.predict(X_valid60)

print('Classification report')
print(classification_report(y_valid60, y_pred))
   
print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid60, y_pred)
print(cf_matrix)

<br>
<font size="3"> <b>KNNeighbors</b> </font>

In [ ]:
modelKnn = KNeighborsClassifier(n_neighbors = 7, weights ='distance')

modelKnn.fit(X_train60, y_train60)
y_pred = modelKnn.predict(X_valid60)

print('Classification report')
print(classification_report(y_valid60, y_pred))

print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid60, y_pred)
print(cf_matrix)

Per quanto riguarda lo splitting, <b>non ci sono alcune differenze nelle performance dei modelli</b>

Ritorno alla fase di "Data preparation" (maggiori dettagli nella documantazione)

<br><br>
<font size="5">Data preparation</font>
<br>

<font size="3"> <b> Feature scaling </b> </font>
<br>
Prima di normalizzare i dati, per rappresentare la diversa distribuzione dei dati, plotto la il valore medio dei dati. 

Dunque guardiamo il valore medio di ogni singola colonna (feature)

In [ ]:
#selezioniamo tutte le colonne che ci interessano in questo caso (ceh sono tutte quelle che hanno un valore numerico)
numeric_cols = finalDataset.select_dtypes(include=['int64','float64']).columns

for feature in numeric_cols:
    print("Media di ",feature,": ",finalDataset[feature].mean())

Già da questo output possiamo vedere che, alcune feature hanno distribuzioni molto diverse rispetto ad altre, guardiamo ad esempio "urlLenght" e "email".
Per avere un idea migliore possiamo plottare i dati: 

In [ ]:
mean_values = []
col_names = []

for col in numeric_cols:
    # calcoliamo la media della colonna
    mean = finalDataset[col].mean()
    mean_values.append(mean)
    col_names.append(col)

#plottiamo il grafico    
fig, ax1 = plt.subplots(figsize=(25, 8), dpi=1000)
ax1.bar(col_names, mean_values)
plt.xlabel('features')
plt.ylabel('Valore medio')
plt.show()

In questa fase veidamo graficamente la diversa distribuzione delle feature. 

<br>
andiamo a normalizzare il dataset utilizzando lo <b> z-score normalization</b>. Ovviamente andremo a normalizzare unicamente le colonne numeriche.

In [ ]:
#seleziono le colonne da normalizzare, quindi selezioni le feature escludendo url e label dalla normalizzazione
columns = ['IP',"urlLenght","ShortiningServ","numDash","numEqual","dash","httpsDomSub","port","numberSub","lenghtDom","lenghtSub","lenghtPath","numLetters","numDigits","email","PunyCode","suspWords","TLD"]
zScoreFinalDataset = finalDataset[columns].apply(zscore)

<br>
Un esempio del risultato della normalizzazione:

In [ ]:
zScoreFinalDataset.head()

Adesso torniamo alla fase di "Modeling" e controlliamo le prestazioni dei modelli dopo la normalizzazione

<br><br>
<font size="5">Modeling</font>

<font size="3"><b>Split dei dati</b></font>

Chiaramente dobbiamo suddividere i dati nuovamente, in quanto dobbiamo allenare e valutare i modelli sui dati normalizzati.
Di conseguenza nella prossima fase andremo a suddividere i dati con la stessa modalità della fase precedente.

In [ ]:
#dataframe contenente tutte le feature
X_norm = zScoreFinalDataset[columns]

#dataframe contenente unicamente la label
y = finalDataset['label']

# definiamo i training set, composto dall'80% dell'intero dataset
X_train_norm, X_rem_norm, y_train_norm, y_rem_norm = train_test_split(X_norm,y, train_size=0.8)

#utilizziamo il dataset rimanente per definire il validation set ed il test set 
X_valid_norm, X_test_norm, y_valid_norm, y_test_norm = train_test_split(X_rem_norm,y_rem_norm, test_size=0.5)
print(X_norm.shape)

<br>
Adesso, esattamente come nella fase di Modeling precedente, andremo nuovamente a ricercare i parametri migliori per ogni singolo modello

<br><font size="3"> <b>Naive Bayes</b> </font>

Cerchiamo la miglior combinazione di parametri per il modello e con il dataset normalizzato

In [ ]:
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6],
    'priors': [None, [0.1, 0.9], [0.2, 0.8], [0.3, 0.7]]
}

# modello gaussiano per dataset normalizzato
modelGaussianNorm = GaussianNB()

# Initialize the grid search
grid_search = GridSearchCV(estimator=modelGaussianNorm, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train_norm, y_train_norm)

# Print the best parameters and the best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Otteremo la stessa lista di parametri del modello precedente ovvero: <i>priors = [0.1, 0.9], var_smoothing = 1e-06 </i> 
Usiamo questi parametri per allenare il modello.

In [ ]:
modelGaussianNorm = GaussianNB(priors = [0.1, 0.9], var_smoothing = 1e-06)

modelGaussianNorm.fit(X_train_norm, y_train_norm)

y_pred = modelGaussianNorm.predict(X_valid_norm)

print('Classification report')
print(classification_report(y_valid_norm, y_pred))

print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid_norm, y_pred)
print(cf_matrix)

<br><font size="3"> <b>Decision Tree</b> </font>

Cerchiamo la miglior combinazione di parametri per il modello e con il dataset normalizzato

In [ ]:
param_grid = {
    'criterion': ["gini", "entropy", "log_loss"],
    'max_depth': [None, 5, 10,7,15],
    'min_samples_split': [2, 5,7,10],
    'min_samples_leaf': [1, 2, 4,6]
}

modelDTNorm = DecisionTreeClassifier()

grid_search = GridSearchCV(estimator=modelDTNorm, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_norm, y_train_norm)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

In questo caso, la migliore combinazione di parametri è <i> criterion ='entropy', max_depth = None, min_samples_leaf =6, min_samples_split = 2 </i> 
Usiamo questi parametri per allenare il modello.

In [ ]:
modelDTNorm = DecisionTreeClassifier(criterion ='entropy', max_depth = None, min_samples_leaf =6, min_samples_split = 2)

modelDTNorm.fit(X_train_norm, y_train_norm)

y_pred = modelDTNorm.predict(X_valid_norm)

print('Classification report')
print(classification_report(y_valid_norm, y_pred))

print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid_norm, y_pred)
print(cf_matrix)

<br><font size="3"> <b>RandomForest</b> </font>

Cerchiamo la miglior combinazione di parametri per il modello e con il dataset normalizzato

In [ ]:
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

modelRFNorm = RandomForestClassifier()

grid_search = GridSearchCV(estimator=modelRFNorm, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_norm, y_train_norm)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Otteremo la stessa lista di parametri : <i> max_depth = None, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 200</i> 
Usiamo questi parametri per allenare il modello.

In [ ]:
modelRFNorm = RandomForestClassifier(max_depth = None, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 200)

modelRFNorm.fit(X_train_norm, y_train_norm)

y_pred = modelRFNorm.predict(X_valid_norm)

print('Classification report')
print(classification_report(y_valid_norm, y_pred))
print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid_norm, y_pred)
print(cf_matrix)

<br><font size="3"> <b>KNeighborsClassifier</b> </font>

Cerchiamo la miglior combinazione di parametri per il modello e con il dataset normalizzato

In [ ]:
param_grid = {'n_neighbors': np.arange(1, 31), 'weights': ['uniform', 'distance']}

modelKnnNorm = KNeighborsClassifier()

grid_search = GridSearchCV(estimator=modelKnnNorm, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_norm, y_train_norm)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Otteremo la stessa lista di parametri: <i> n_neighbors = 8, weights = 'distance' </i> 
Usiamo questi parametri per allenare il modello.

In [ ]:
modelKnnNorm = KNeighborsClassifier(n_neighbors = 8, weights = 'distance')

modelKnnNorm.fit(X_train_norm, y_train_norm)
y_pred = modelKnnNorm.predict(X_valid_norm)

print('Classification report')
print(classification_report(y_valid_norm, y_pred))

print('Confusion_matrix')
cf_matrix = confusion_matrix(y_valid_norm, y_pred)
print(cf_matrix)


Come è possibile vedere dalle metriche dei modelli, la normalizzazione del dataset <b>non ha portato un aumento delle prestazioni</b>, anzi, in alcuni modelli come il GaussianNB ha portato ad un degrado delle prestazioni. 

Di conseguenza sceglieremo il modello <i>RandomForestClassifier</i> utilizzando i parametri <i>max_depth = None, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 200 </i> allenato su dati non normalizzati.

<br><br>
<font size="5">Evaluation</font>


In questa fase andremo a valutare l’accuratezza del modello.

<br>
<font size="3"><b>Valutazione del modello</b></font>

Dato che la normalizzazione non ha portato miglioramenti nella prestazione dei modelli, utilizzo il modello allenato su dati non normalizzati(ovvero modelRF).
Nella prossima fase andremo a validare il modello con gli insiemi di test non normalizzati(X_test e y_test) prodotti in fase di "Modeling".

In [ ]:
#modelRF è il modello Random Forest Classifier allenato su dati non normalizzati
y_prediction = modelRF.predict(X_test)

print('Classification report')
print(classification_report(y_test, y_prediction))
print('Confusion_matrix')
cf_matrix = confusion_matrix(y_test, y_prediction)
print(cf_matrix)

Otterremo dunque il nostro modello finale. 

Dalla fase di "Modeling" possiamo passara a due diverse fasi. Nel caso in cui gli obbiettivi di business non sono stati raggiunti, è necessario tornare alla fase di "Business understanding" per ricominciare il ciclo.

Nel caso in cui invece tutti gli obbiettivi di business sono stati raggiunti possiamo passare alla fase di "Deployment". 

<br><br>
<font size="5">Deployment</font>

Per la fase di deployment, decido di creare un piccolo applicativo capace di utilizzare il modello appena creato.

L'applicazione non verrà sviluppata in questo notebook, ma è possibile trovarla nel file <i>Demo.py</i>

<br>
<font size="3"> <b>salva il modello</b> </font>
<br>
Salviamo il modello in un file .joblib, ed utilizzeremo questo file per caricare il modello nell'applicazione.

In [ ]:
from joblib import dump, load
dump(modelRF, 'modelRF.joblib')

In [ ]:
infine vorrei aggiungere un ultima osservazione RIGUARDO FEAURE VS TUTTE LE FEATURE 